In [1]:
import os, sys
notebook_dir = os.getcwd()
sys.path.append(os.path.abspath(os.path.join(notebook_dir,'..')))

In [8]:
from scripts.data_load import *
import re

In [4]:
df = load_data_excel('../Data/telegram_data.xlsx')

In [5]:
print("Checking for NaN values in the 'Message' column:")
nan_count = df['Message'].isnull().sum()
print(f"Number of NaN values in 'Message' column: {nan_count}")

Checking for NaN values in the 'Message' column:
Number of NaN values in 'Message' column: 1849


In [6]:
df = df.dropna(subset=['Message'])

# Print the shape of the dataset after dropping NaN values in the "Message" column
print(f"Dataset shape after dropping NaN values in 'Message' column: {df.shape}")

Dataset shape after dropping NaN values in 'Message' column: (3166, 6)


In [7]:
message_df=df['Message']
message_df

5       💥3pcs silicon brush spatulas\n\n⚡እስከ 260°c ሙቀት...
6       💥Mandoline Slicer\n\n👉 ጊዜ ቆጣቢ ስላይስ ማድረጊያ \n👉  ...
7       💥Table Desk Edge Guard Strip\n       💯 High Qu...
8       💥Table Desk Edge Guard Strip\n       💯 High Qu...
10      💥Only baby 3in1 double bottle milk warmer,ster...
                              ...                        
5009    🎯 Kitchen Sticker\n\nለኪችንዎ ውበት እጅግ ተመራጭ \n🔰ውሀ ...
5010    🎯 3in1 One Step Hair Dryer & Styler \n\n👉 ከርል ...
5011    ✅ Home GYM - X5 slimming vibrator \n\n📢📢📢 ታላቅ ...
5012    ለጤናችን-Health & Personal Care\n\n📍FingerTip Pul...
5013    #Finger_tip_pulse_oximeter\n       #በተመጣጣኝ_ዋጋ\...
Name: Message, Length: 3166, dtype: object

In [9]:
def remove_emojis(text):
    emoji_pattern = re.compile(
        "[" 
        "\U0001F600-\U0001F64F"  # emoticons
        "\U0001F300-\U0001F5FF"  # symbols & pictographs
        "\U0001F680-\U0001F6FF"  # transport & map symbols
        "\U0001F700-\U0001F77F"  # alchemical symbols
        "\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
        "\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
        "\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
        "\U0001FA00-\U0001FA6F"  # Chess Symbols
        "\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
        "\U00002702-\U000027B0"  # Dingbats
        "\U000024C2-\U0001F251" 
        "]+", 
        flags=re.UNICODE
    )
    return emoji_pattern.sub(r'', text)

# Apply the function to the 'Message' column
df['Message'] = df['Message'].apply(remove_emojis)

# Display the updated DataFrame
print(df.head())

          Channel Title    Channel Username    ID  \
5   Sheger online-store  @Shageronlinestore  5328   
6   Sheger online-store  @Shageronlinestore  5327   
7   Sheger online-store  @Shageronlinestore  5326   
8   Sheger online-store  @Shageronlinestore  5325   
10  Sheger online-store  @Shageronlinestore  5323   

                                              Message  \
5   3pcs silicon brush spatulas\n\nእስከ 260°c ሙቀት መ...   
6   Mandoline Slicer\n\n ጊዜ ቆጣቢ ስላይስ ማድረጊያ \n  ለእጅ...   
7   Table Desk Edge Guard Strip\n        High Qual...   
8   Table Desk Edge Guard Strip\n        High Qual...   
10  Only baby 3in1 double bottle milk warmer,steri...   

                         Date                          Media Path  
5   2024-09-20 11:50:02+00:00  photos/@Shageronlinestore_5328.jpg  
6   2024-09-20 08:11:40+00:00                                 NaN  
7   2024-09-20 05:23:18+00:00                                 NaN  
8   2024-09-20 05:21:14+00:00  photos/@Shageronlinestore_5325.jpg 

In [10]:
df.to_csv('../Data/clean_data.csv')

In [11]:
def label_message_utf8_with_birr(message):
    # Split the message at the first occurrence of '\n'
    if '\n' in message:
        first_line, remaining_message = message.split('\n', 1)
    else:
        first_line, remaining_message = message, ""
    
    labeled_tokens = []
    
    # Tokenize the first line
    first_line_tokens = re.findall(r'\S+', first_line)
    
    # Label the first token as B-PRODUCT and the rest as I-PRODUCT
    if first_line_tokens:
        labeled_tokens.append(f"{first_line_tokens[0]} B-PRODUCT")  # First token as B-PRODUCT
        for token in first_line_tokens[1:]:
            labeled_tokens.append(f"{token} I-PRODUCT")  # Remaining tokens as I-PRODUCT
    
    # Process the remaining message normally
    if remaining_message:
        lines = remaining_message.split('\n')
        for line in lines:
            tokens = re.findall(r'\S+', line)  # Tokenize each line while considering non-ASCII characters
            
            for token in tokens:
                # Check if token is a price (e.g., 500 ETB, $100, or ብር)
                if re.match(r'^\d{10,}$', token):
                    labeled_tokens.append(f"{token} O")  # Label as O for "other" or outside of any entity
                elif re.match(r'^\d+(\.\d{1,2})?$', token) or 'ETB' in token or 'ዋጋ' in token or '$' in token or 'ብር' in token:
                    labeled_tokens.append(f"{token} I-PRICE")
                # Check if token could be a location (e.g., cities or general location names)
                elif any(loc in token for loc in ['Addis Ababa', 'ለቡ', 'ለቡ መዳህኒዓለም', 'መገናኛ', 'ቦሌ', 'ሜክሲኮ']):
                    labeled_tokens.append(f"{token} I-LOC")
                # Assume other tokens are part of a product name or general text
                else:
                    labeled_tokens.append(f"{token} O")
    
    return "\n".join(labeled_tokens)

# Apply the updated function to the non-null messages
df['Labeled_Message'] = df['Message'].apply(label_message_utf8_with_birr)

# Display the updated DataFrame
df.head()


,Channel Title,Channel Username,ID,Message,Date,Media Path,Labeled_Message
5,Sheger online-store,@Shageronlinestore,5328,3pcs silicon brush spatulas\n\nእስከ 260°c ሙቀት መ...,2024-09-20 11:50:02+00:00,photos/@Shageronlinestore_5328.jpg,3pcs B-PRODUCT\nsilicon I-PRODUCT\nbrush I-PRO...
6,Sheger online-store,@Shageronlinestore,5327,Mandoline Slicer\n\n ጊዜ ቆጣቢ ስላይስ ማድረጊያ \n ለእጅ...,2024-09-20 08:11:40+00:00,NaN,Mandoline B-PRODUCT\nSlicer I-PRODUCT\nጊዜ O\nቆ...
7,Sheger online-store,@Shageronlinestore,5326,Table Desk Edge Guard Strip\n High Qual...,2024-09-20 05:23:18+00:00,NaN,Table B-PRODUCT\nDesk I-PRODUCT\nEdge I-PRODUC...
8,Sheger online-store,@Shageronlinestore,5325,Table Desk Edge Guard Strip\n High Qual...,2024-09-20 05:21:14+00:00,photos/@Shageronlinestore_5325.jpg,Table B-PRODUCT\nDesk I-PRODUCT\nEdge I-PRODUC...
10,Sheger online-store,@Shageronlinestore,5323,"Only baby 3in1 double bottle milk warmer,steri...",2024-09-19 13:54:46+00:00,photos/@Shageronlinestore_5323.jpg,Only B-PRODUCT\nbaby I-PRODUCT\n3in1 I-PRODUCT...


In [12]:
labeled_data_birr_path = 'labeled_telegram_product_price_location.txt-'
with open(labeled_data_birr_path, 'w', encoding='utf-8') as f:
    for index, row in df.iterrows():
        f.write(f"{row['Labeled_Message']}\n\n")